In [94]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import sys
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.model_selection import train_test_split
import platform

In [95]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

total_class = len(class_names)

In [96]:
# Parameters
n_inputs = 28 ### lengh of each row
n_steps = 28 ### number of time steps
n_layers = 3 ### number of BasicRNNCell layers
n_neurons = 100 ### number of neurons in the network
n_outputs = 10 ### outputs that represent digits from 0-9
training_epochs = 100
learning_rate = 0.001
batch_size = 50
patience = int(np.sqrt(training_epochs))

In [97]:
def reset_graph(seed=42): ### it resests all created graph, it's required once re-defining of any placeholders, variables, shapes or model structures is needed
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [98]:
def generate_batch(x_train, y_train, batch_size): ### it takes random permutation of lenght x_train and splits x_train (together with y_train) into batches number
    rnd_idx = np.random.permutation(len(x_train))
    n_batches = len(x_train) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        x_batch, y_batch = x_train[batch_idx], y_train[batch_idx]
        yield x_batch, y_batch

In [99]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data() ### loading the datasets
x_train = x_train.astype(np.float32).reshape(-1, 28*28) / 255.0 ### reshaping and normalizing
x_test = x_test.astype(np.float32).reshape(-1, 28*28) / 255.0 ### reshaping and normalizing
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [100]:
reset_graph()

In [101]:
with tf.name_scope("Inputs"):
    X = tf.placeholder(tf.float32, [None, n_steps, n_inputs], name="X")
    y = tf.placeholder(tf.int32, [None], name="y")
    keep_prob = tf.placeholder_with_default(1.0, shape=(), name='keep_probability')

Tensor("Inputs/X:0", shape=(?, 28, 28), dtype=float32)
Tensor("Inputs/y:0", shape=(?,), dtype=int32)
Tensor("Inputs/keep_probability:0", shape=(), dtype=float32)


In [102]:
with tf.name_scope("Basic_RNN_Layers"):
    lstm_cells = [tf.nn.rnn_cell.BasicRNNCell(num_units = n_neurons, activation = tf.nn.relu)
             for layer in range(n_layers)]
    lstm_cells_drop = [tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=keep_prob)
                for cell in lstm_cells]
    multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell(lstm_cells_drop)
    outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype = tf.float32) ### states return final state (last output) of the multi_layer_cell

/home/sauravkoranga/.local/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:421: UserWarning: `tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.SimpleRNNCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be "
/home/sauravkoranga/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [103]:
with tf.name_scope("Loss"):
    states_concat = tf.concat(axis=1, values=states, name='states_reshape')
    dense1 = tf.layers.dense(states_concat, 64, name='dense_1')
    dense2 = tf.layers.dense(dense1, 32, name='dense_2')
    logits = tf.layers.dense(dense2, n_outputs, name='output_layer')
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=tf.reshape(logits, shape=(-1, n_outputs)), name='softmax_cross_entropy')
    loss = tf.reduce_mean(xentropy, name='loss')
    loss_summary = tf.summary.scalar('loss_summ', loss)

/home/sauravkoranga/.local/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/home/sauravkoranga/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [104]:
with tf.name_scope("Train"):    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, name='Adam_optimizer')
    training_optimizer = optimizer.minimize(loss, name='training_Adam')

In [105]:
with tf.name_scope("Evaluation"):        
    correct = tf.nn.in_top_k(tf.reshape(logits, (-1, n_outputs)), y, 1, name='inTopK')
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='Accuracy')
    accuracy_summary = tf.summary.scalar('Accuracy_Summ', accuracy)

In [106]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
train_keep_prob = 0.8
x_test = x_test.reshape((-1, n_steps, n_inputs)) ### reshaping test set

In [108]:
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime('%Y-%m-%d-%H-%m-%S')
    root_logdir = "TensorFlow_Logs"
    if prefix:
        prefix += '-'
    name = prefix + now
    return '{}/{}/'.format(root_logdir, name)


In [109]:
logdir=log_dir("mnist_rnn_model")
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [115]:
checkpoint_path = "./checkpoints/mnist_rnn_model.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./mnist_rnn_model"

In [126]:
def train():
    best_loss = np.infty                ### parameters for early stopping
    epochs_without_progress = 0         ### once epochs_without_progress reaches the value
    max_epochs_without_progress = 15    ### of max_epochs_without_progress, the model stops and saves last parameters


    acc_list, acc_test_list, loss_list, loss_test_list = [], [], [], []
    n_epochs = 400
    batch_size = 128
    with tf.Session() as sess:
        init.run() 
        for epoch in range(n_epochs):
            for x_batch, y_batch in generate_batch(x_train, y_train, batch_size):
                x_batch = x_batch.reshape((-1, n_steps, n_inputs))

                sess.run(training_optimizer, feed_dict={X: x_batch, y: y_batch, keep_prob: train_keep_prob})
            acc_batch, loss_batch, acc_sum, loss_sum = sess.run([accuracy, loss, accuracy_summary, loss_summary], feed_dict={X: x_batch, y: y_batch, keep_prob: train_keep_prob})   
   
            acc_test, loss_test, acc_test_sum, loss_test_sum = sess.run([accuracy, loss, accuracy_summary, loss_summary], feed_dict={X: x_test, y: y_test})

            acc_list.append(acc_batch)
            loss_list.append(loss_batch)
            acc_test_list.append(acc_test)
            loss_test_list.append(loss_test)

            file_writer.add_summary(acc_sum, epoch)
            file_writer.add_summary(loss_sum, epoch)
            file_writer.add_summary(acc_test_sum, epoch)
            file_writer.add_summary(loss_test_sum, epoch)

            if epoch % 5 == 0:
                print("Epoch", epoch,
                      '\tValidation accuracy: {:.3f}%'.format(acc_batch * 100), '\tLoss: {:.3f}'.format(loss_batch))
                saver.save(sess, checkpoint_path)
                with open(checkpoint_epoch_path, "wb") as f:
                    f.write(b'%d' % (epoch + 1))
                if loss_batch < best_loss:
                    saver.save(sess, final_model_path)
                    best_loss = loss_batch
                else:
                    epochs_without_progress += 2
                    if epochs_without_progress > max_epochs_without_progress:
                        print('Early Stopping')
                        break

In [127]:
def test():
    best_loss = np.infty
    epochs_without_progress = 0
    max_epochs_without_progress = 15


    acc_list, acc_test_list, loss_list, loss_test_list = [], [], [], []
    n_epochs = 400
    batch_size = 128
    with tf.Session() as sess:
        init.run() 
        for epoch in range(n_epochs):
            for x_batch, y_batch in generate_batch(x_train, y_train, batch_size):
                x_batch = x_batch.reshape((-1, n_steps, n_inputs))

                sess.run(training_optimizer, feed_dict={X: x_batch, y: y_batch, keep_prob: train_keep_prob})
            acc_batch, loss_batch, acc_sum, loss_sum = sess.run([accuracy, loss, accuracy_summary, loss_summary], feed_dict={X: x_batch, y: y_batch, keep_prob: train_keep_prob})   
   
            acc_test, loss_test, acc_test_sum, loss_test_sum = sess.run([accuracy, loss, accuracy_summary, loss_summary], feed_dict={X: x_test, y: y_test})

            acc_list.append(acc_batch)
            loss_list.append(loss_batch)
            acc_test_list.append(acc_test)
            loss_test_list.append(loss_test)  

            file_writer.add_summary(acc_sum, epoch)
            file_writer.add_summary(loss_sum, epoch)
            file_writer.add_summary(acc_test_sum, epoch)
            file_writer.add_summary(loss_test_sum, epoch)

            if epoch % 5 == 0:
                print("Epoch", epoch, '\tTest accuracy: {:.3f}%'.format(acc_test * 100), '\tLoss: {:.3f}'.format(loss_batch))
                saver.save(sess, checkpoint_path)
                with open(checkpoint_epoch_path, "wb") as f:
                    f.write(b'%d' % (epoch + 1))
                if loss_batch < best_loss:
                    saver.save(sess, final_model_path)
                    best_loss = loss_batch
                else:
                    epochs_without_progress += 2
                    if epochs_without_progress > max_epochs_without_progress:
                        print('Early Stopping')
                        break

In [128]:
train()

Epoch 0 	Validation accuracy: 82.812% 	Loss: 0.464
Epoch 5 	Validation accuracy: 86.719% 	Loss: 0.393
Epoch 10 	Validation accuracy: 82.812% 	Loss: 0.381
Epoch 15 	Validation accuracy: 96.094% 	Loss: 0.212
Epoch 20 	Validation accuracy: 89.062% 	Loss: 0.355
Epoch 25 	Validation accuracy: 87.500% 	Loss: 0.381
Epoch 30 	Validation accuracy: 93.750% 	Loss: 0.190
Epoch 35 	Validation accuracy: 91.406% 	Loss: 0.201
Epoch 40 	Validation accuracy: 92.969% 	Loss: 0.240
Epoch 45 	Validation accuracy: 87.500% 	Loss: 0.317
Epoch 50 	Validation accuracy: 91.406% 	Loss: 0.176
Epoch 55 	Validation accuracy: 90.625% 	Loss: 0.288
Epoch 60 	Validation accuracy: 91.406% 	Loss: 0.217
Epoch 65 	Validation accuracy: 84.375% 	Loss: 0.333
Early Stopping
